# Imports

In [1]:
import matplotlib.pyplot as plt
from matplotlib.image import imread
import os
from os import makedirs, listdir
import shutil
from shutil import copyfile
import numpy as np
import pandas as pd
from random import seed
import io

# Plotting images

In [ ]:
# plot lion photos
folder = './images/lion/'

fig, ax = plt.subplots(3, 3, figsize=(10, 10)) 
for i in range(9):
	filename = os.path.join(folder, os.listdir(folder)[i])
	ax[i//3, i%3].imshow(imread(filename))

plt.show()

In [ ]:
imread(os.path.join('./images/kangaroo/', os.listdir('./images/kangaroo/')[0])).shape

In [ ]:
# plot kangaroo photos
folder = './images/kangaroo/'

fig, ax = plt.subplots(3, 3, figsize=(10, 10))
for i in range(9):
	filename = os.path.join(folder, os.listdir(folder)[i])
	ax[i//3, i%3].imshow(imread(filename))

plt.show()

# Splitting Into train and test (80-20)

In [ ]:
shutil.rmtree('dataset/', ignore_errors=True)
dataset_home = 'dataset/'
subdirs = ['train/', 'test/']
for subdir in subdirs:
	# create label subdirectories
	labeldirs = ['kangaroo/', 'lion/']
	for labldir in labeldirs:
		newdir = dataset_home + subdir + labldir
		makedirs(newdir, exist_ok=True)

In [ ]:
seed(42)
val_ratio = 20
testlist = np.random.choice(range(100), val_ratio, replace=False)

# copy training dataset images into subdirectories
src_directory = 'images/kangaroo/'
files = listdir(src_directory)
for i in range(100):
	file = files[i]
	src = src_directory + '/' + file
	dst = ('dataset/test/kangaroo/' + file) if i in testlist else ('dataset/train/kangaroo/' + file)
	copyfile(src, dst)

In [ ]:
seed(42)
val_ratio = 20
testlist = np.random.choice(range(100), val_ratio, replace=False)

# copy training dataset images into subdirectories
src_directory = 'images/lion/'
files = listdir(src_directory)
for i in range(100):
	file = files[i]
	src = src_directory + '/' + file
	dst = ('dataset/test/lion/' + file) if i in testlist else ('dataset/train/lion/' + file)
	copyfile(src, dst)

# Loading Images

In [2]:
from keras.applications.vgg16 import VGG16
from keras.models import Sequential, Model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import timeit

%load_ext tensorboard

In [3]:
# create data generator
datagen = ImageDataGenerator(rescale=1.0/255.0)

target_size = (150, 150)
# prepare iterators
train_set = datagen.flow_from_directory('dataset/train/', class_mode='binary', batch_size=64, target_size=target_size)
test_set = datagen.flow_from_directory('dataset/test/', class_mode='binary', batch_size=64, target_size=target_size)

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


# Models

## VGG1

In [4]:
# VGG 1 block
def VGG1():
	model = Sequential(name='VGG1')
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(target_size[0], target_size[1], 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = SGD(learning_rate=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

## VGG3

In [5]:
# VGG 3 blocks
def VGG3():
	model = Sequential(name='VGG3')
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(target_size[0], target_size[1], 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = SGD(learning_rate=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

## VGG 16

In [6]:
# VGG 16 Model
def VGG_16():
   # load model
   model = VGG16(include_top=False, input_shape=(target_size[0], target_size[1], 3))
   # mark loaded layers as not trainable
   for layer in model.layers:
      layer.trainable = False
   # add new classifier layers
   flat1 = Flatten()(model.layers[-1].output)
   class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
   output = Dense(1, activation='sigmoid')(class1)
   # define new model
   model = Model(inputs=model.inputs, outputs=output, name='VGG16')
   # compile model
   opt = SGD(learning_rate=0.001, momentum=0.9)
   model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
   return model

## Utils

In [ ]:
# plot diagnostic learning curves
def summarize_diagnostics(history, name):
	fig, ax = plt.subplots(2, 1, figsize=(10, 10))	

	ax[0].plot(history.history['loss'], color='blue', label='train')
	ax[0].plot(history.history['val_loss'], color='orange', label='test')
	ax[0].set_title('Cross Entropy Loss')
	ax[0].set_xlabel('Epochs')
	ax[0].set_ylabel('Loss')
	ax[0].legend()

	ax[1].plot(history.history['accuracy'], color='blue', label='train')
	ax[1].plot(history.history['val_accuracy'], color='orange', label='test')
	ax[1].set_title('Classification Accuracy')
	ax[1].set_xlabel('Epochs')
	ax[1].set_ylabel('Accuracy')
	ax[1].legend()
	
	# save plot to file
	plt.savefig('plots/' + name + '_plot.png')
	plt.close()

In [7]:
def plot_to_image(figure):
  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=4)
  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image

def image_grid(imgs, preds):
  """Return a 5x5 grid of the MNIST images as a matplotlib figure."""
  figure = plt.figure(figsize=(10,10))
  for i in range(40):
    # Start next subplot.
    plt.subplot(8, 5, i + 1, title=preds[i])
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(imgs[i])

  return figure

In [8]:
# makedirs('plots', exist_ok=True)

df = pd.DataFrame(columns=['Training time', 'Training loss', 'Training accuracy', 'Testing accuracy', 'Number of model parameters'], index=["VGG1", "VGG3", "VGG3_augmented", "VGG16"])

In [25]:
# shutil.rmtree('logs', ignore_errors=True)
shutil.rmtree('logs/fit/VGG3_augmented/', ignore_errors=True)
shutil.rmtree('logs/predictions/VGG3_augmented/', ignore_errors=True)

# Training

Model is trained one by one and not with a for loop due Resource exhausting error.

In [ ]:
models = [VGG1, VGG3, VGG_16]
# for model in models:
with tf.device('/device:CPU:0'):
	model = VGG_16()

	tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/fit/" + model.name, update_freq='batch')

	start = timeit.default_timer()
	# fit model
	history = model.fit(train_set, steps_per_epoch=len(train_set), validation_data=test_set, validation_steps=len(test_set), epochs=20, verbose=0, callbacks=[tensorboard_callback])
	exec_time = timeit.default_timer() - start

	predictions = np.where((model.predict(test_set[0][0]) > 0.5).astype("int32").reshape(-1) == 1, 'lion', 'kangaroo')

	file_writer = tf.summary.create_file_writer("logs/predictions/" + model.name)

	figure = image_grid(test_set[0][0], predictions)
	with file_writer.as_default():
		tf.summary.image("Training data", plot_to_image(figure), step=0)

	# evaluate model
	_, acc = model.evaluate(test_set, steps=len(test_set), verbose=0)
	print('> %.3f' % (acc * 100.0))

	df.loc[model.name, 'Training time'] = exec_time
	df.loc[model.name, 'Testing accuracy'] = acc * 100.0
	df.loc[model.name, 'Training accuracy'] = history.history['accuracy'][-1] * 100.0
	df.loc[model.name, 'Training loss'] = history.history['loss'][-1]
	df.loc[model.name, 'Number of model parameters'] = model.count_params()
		
		# learning curves
		# summarize_diagnostics(history, model.name)


In [ ]:
predictions = np.where((model.predict(test_set[0][0]) > 0.5).astype("int32").reshape(-1) == 1, 'lion', 'kangaroo')
predictions

# VGG 3 with Data Augmentation

In [27]:
# create data generators
aug_train_datagen = ImageDataGenerator(rescale=1.0/255.0, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
aug_test_datagen = ImageDataGenerator(rescale=1.0/255.0)

aug_train_set = aug_train_datagen.flow_from_directory('dataset/train/',class_mode='binary', batch_size=64, target_size=target_size)
aug_test_set = aug_test_datagen.flow_from_directory('dataset/test/',class_mode='binary', batch_size=64, target_size=target_size) 

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [28]:
with tf.device('/device:CPU:0'):
	model = VGG3()

	tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/fit/VGG3_augmented", update_freq="batch")

	start = timeit.default_timer()
	history = model.fit(aug_train_set, steps_per_epoch=len(aug_train_set), validation_data=aug_test_set, validation_steps=len(aug_test_set), epochs=20, verbose=0, callbacks=[tensorboard_callback])
	exec_time = timeit.default_timer() - start
	# evaluate model
	_, acc = model.evaluate(aug_test_set, steps=len(aug_test_set), verbose=0)
	print('> %.3f' % (acc * 100.0))

	predictions = np.where((model.predict(aug_test_set[0][0]) > 0.5).astype("int32").reshape(-1) == 1, 'lion', 'kangaroo')

	file_writer = tf.summary.create_file_writer("logs/predictions/" + model.name + "_augmented")

	figure = image_grid(aug_test_set[0][0], predictions)

	with file_writer.as_default():
		tf.summary.image("Training data", plot_to_image(figure), step=0)

	df.loc[model.name + "_augmented", 'Training time'] = exec_time
	df.loc[model.name + "_augmented", 'Testing accuracy'] = acc * 100.0
	df.loc[model.name + "_augmented", 'Training accuracy'] = history.history['accuracy'][-1] * 100.0
	df.loc[model.name + "_augmented", 'Training loss'] = history.history['loss'][-1]
	df.loc[model.name + "_augmented", 'Number of model parameters'] = model.count_params()

# # learning curves
# summarize_diagnostics(history, model.name + "_augmented")

> 77.500
2/2 [==============================] - 0s 37ms/step


# MLP 

In [21]:
def our_MLP():
    model = Sequential(name='our_MLP')
    model.add(Flatten(input_shape=(target_size[0], target_size[1], 3)))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(2048, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [22]:
with tf.device('/device:GPU:0'):
	model = our_MLP()

	start = timeit.default_timer()
	history = model.fit(train_set, steps_per_epoch=len(train_set), validation_data=test_set, validation_steps=len(test_set), epochs=10, verbose=0)
	exec_time = timeit.default_timer() - start
	# evaluate model
	_, acc = model.evaluate(test_set, steps=len(test_set), verbose=0)
	print('> %.3f' % (acc * 100.0))

	df.loc[model.name, 'Training time'] = exec_time
	df.loc[model.name, 'Testing accuracy'] = acc * 100.0
	df.loc[model.name, 'Training accuracy'] = history.history['accuracy'][-1] * 100.0
	df.loc[model.name, 'Training loss'] = history.history['loss'][-1]
	df.loc[model.name, 'Number of model parameters'] = model.count_params()

	# learning curves
	# summarize_diagnostics(history, model.name)

> 52.500


# Results

In [ ]:
%tensorboard --logdir logs/fit

In [29]:
df

,Training time,Training loss,Training accuracy,Testing accuracy,Number of model parameters
VGG1,10.173842,0.431681,83.749998,60.000002,23041153
VGG3,68.59395,0.464399,74.374998,69.999999,5401921
VGG3_augmented,65.54307,0.550922,75.625002,77.499998,5401921
VGG16,325.50215,0.148732,96.875,94.999999,15763521
our_MLP,4.862237,0.737315,50.625002,52.499998,73974785


In [30]:
# save pandas dataframe to csv
df.to_csv('results.csv')

In [31]:
# load pandas dataframe from csv
ldf = pd.read_csv('results.csv', index_col=0)
ldf

,Training time,Training loss,Training accuracy,Testing accuracy,Number of model parameters
VGG1,10.173842,0.431681,83.749998,60.000002,23041153
VGG3,68.593950,0.464399,74.374998,69.999999,5401921
VGG3_augmented,65.543070,0.550922,75.625002,77.499998,5401921
VGG16,325.502150,0.148732,96.875000,94.999999,15763521
our_MLP,4.862237,0.737315,50.625002,52.499998,73974785
